In [1]:
#pip install tensorflow==1.15
#Install stable-baselines as described in the documentation

import gym
import gym_pcgrl
from gym_pcgrl import wrappers

from stable_baselines.common.policies import MlpPolicy, CnnPolicy, FeedForwardPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines.a2c.utils import conv, linear, conv_to_fc
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines import PPO2

import tensorflow as tf
import numpy as np

import pdb

ImportError: No module named 'stable_baselines'

# Load Random Model

In [2]:
def Cnn(image, **kwargs):
    activ = tf.nn.relu
    layer_1 = activ(conv(image, 'c1', n_filters=32, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs)) # filter_size=3
    layer_2 = activ(conv(layer_1, 'c2', n_filters=64, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs)) #filter_size = 3
    layer_3 = activ(conv(layer_2, 'c3', n_filters=64, filter_size=3, stride=1, init_scale=np.sqrt(2), **kwargs))
    layer_3 = conv_to_fc(layer_3)
    return activ(linear(layer_3, 'fc1', n_hidden=512, init_scale=np.sqrt(2)))

class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs, cnn_extractor=Cnn, feature_extraction="cnn")

model = PPO2(CustomPolicy, env, verbose=1, tensorboard_log="./runs")

NameError: name 'FeedForwardPolicy' is not defined

# Load Saved Model

In [3]:
model = PPO2.load("binary_narrow_limited_centered")

NameError: name 'PPO2' is not defined

# Run Model

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display

def show_state(env, l, c, r, step=0, name="", info=""):
    fig = plt.figure(10)
    plt.clf()
    
    plt.title("{} | Step: {} Path: {} Changes: {} Regions: {}".format(name, step, l[-1], c[-1], r[-1]))
    
    ax1 = fig.add_subplot(1,4,1)
    ax1 = plt.imshow(env.render(mode='rgb_array'))
    plt.axis('off')
    
    ax2 = fig.add_subplot(1,4,2)
    ax2 = plt.plot(l)
    
    ax3 = fig.add_subplot(1,4,3)
    ax3 = plt.plot(c)
    
    ax4 = fig.add_subplot(1,4,4)
    ax4 = plt.plot(r)
    
    
    fig.set_figwidth(15)
    plt.tight_layout()
    
    display.clear_output(wait=True)
    display.display(plt.gcf())

def get_env():
    env = wrappers.CroppedImagePCGRLWrapper("sokoban-narrow-v0", 12, random_tile=False)
    return env

def get_action(env, model, action_type=True):
    action = None
    if action_type == 0:
        action, _ = model.predict(obs)
    elif action_type == 1:
        action_prob = model.action_probability(obs)[0]
        action = np.random.choice(a=list(range(len(action_prob))), size=1, p=action_prob)
    else:
        action = np.array([env.action_space.sample()])
    return action

env = DummyVecEnv([get_env])
# Enjoy trained agent
path_length = []
changes = []
regions = []
obs = env.reset()
for i in range(1000):
    action = get_action(env, model, 1)
    obs, rewards, dones, info = env.step(action)
    path_length.append(info[0]['path-length'])
    changes.append(info[0]['changes'])
    regions.append(info[0]['regions'])
    if dones:
        break
    show_state(env, path_length, changes, regions, i)

NameError: name 'DummyVecEnv' is not defined